In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simsopt
import simsopt.geo
import simsopt.field
import os

from bdistrib_util import *

In [ ]:
def get_path(ID, type="simsopt"):
  fID = ID // 1000 
  if type == "simsopt":
    return f'./QUASR_db/simsopt_serials/{fID:04}/serial{ID:07}.json'
  elif(type == "nml"):
    return f'./QUASR_db/nml/{fID:04}/input{ID:07}'
  elif(type == "bdistrib"):
    return f'./QUASR_db/bdistrib_serials/{fID:04}/bdistrib_out.{ID:07}.nc'
  else:
    raise RuntimeError()

sopt_objs = []
for i in range(1000):
  try:
    sopt_objs.append(simsopt.load(get_path(i)))
  except:
    pass
  # simsopt.geo.plot(np.concatenate(sopt_objs[-1]))
  
df = pd.DataFrame(sopt_objs, columns=["surfaces", "coils"])
df["lcfs"] = df["surfaces"].map(lambda x: x[-1])
df["AR"] = df["lcfs"].map(lambda x: x.aspect_ratio())
df["volume"] = df["lcfs"].map(lambda x: -x.volume())
df["nfp"] = df["lcfs"].map(lambda x: x.nfp)
df["R1"] = df["lcfs"].map(lambda x: x.minor_radius())

print(len(df))
df.head()

In [ ]:
def eval_row(row, plot=False):
  coils = row["coils"]
  nfp = row["nfp"]
  R1 = row["R1"]
  lcfs = row["lcfs"]
  computational_surface = simsopt.geo.SurfaceRZFourier.from_nphi_ntheta(nfp=nfp, nphi=64, ntheta=64, range="half period")

  # Simple torus boundary
  computational_surface.set_rc(1,0,R1*2.0)
  computational_surface.set_zs(1,0,R1*2.0)
  computational_surface.change_resolution(3,4)
  # computational_surface.set_rc(2,1,np.random.rand())
  # computational_surface.set_zs(1,2,np.random.rand())
  normal_computational = computational_surface.normal()
  xyz_computational = computational_surface.gamma()

  # Scaled computational boundary
  scale = 1.5
  lcfs.scale(scale)
  xyz_computational = lcfs.gamma()
  normal_computational = lcfs.normal()
  lcfs.scale(1.0 / scale)

  bs = simsopt.field.BiotSavart(coils)
  bs.set_points_cart(xyz_computational.reshape((-1,3)))
  B = bs.B().reshape(xyz_computational.shape)
  BdotN = np.sum(normal_computational * B, axis=-1)
  BdotN_fft = np.fft.fft2(BdotN)
  
  if plot:
    import plotly.express as px
    # px.imshow(BdotN).show()
    # px.imshow(np.abs(np.fft.fftshift(BdotN_fft))).show()
    # px.imshow(np.real(np.fft.fftshift(BdotN_fft)), title="Real component").show()
    # px.imshow(np.imag(np.fft.fftshift(BdotN_fft)), title="Imag component").show()
    plt.figure(figsize=(16,5))
    plt.subplot(131)
    plt.imshow(BdotN)
    plt.title("BdotN")
    plt.colorbar()
    plt.subplot(132)
    plt.imshow(np.real(np.fft.fftshift(BdotN_fft)))
    plt.title("fft real")
    plt.colorbar()
    plt.subplot(133)
    plt.imshow(np.imag(np.fft.fftshift(BdotN_fft)))
    plt.title("fft imag")
    plt.colorbar()
    # plt.tight_layout()
    plt.show()

  return BdotN_fft

for idx,row in df.iterrows():
  eval_row(row, False)

- Check simple computational boundaries
- Check using a simple scaled up boundary
- Is SVD independent of boundary geometry?

In [ ]:
dfid = 3
lcfs = df["lcfs"][dfid]
curves = [coil.curve for coil in df["coils"][dfid]]

# XYZ tensor fourier -> RZ fourier
rzf = lcfs.to_RZFourier()
rzf.plot()

In [ ]:
import subprocess
subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf), 
                                                                sep_outer=minimum_coil_surf_distance(curves, rzf))])
import bdistribPlot
bdistribPlot.main("bdistrib_out.python_generated.nc")

In [ ]:
def get_LgradB(path):
  from simsopt.util import MpiPartition
  from simsopt.mhd import Vmec
  
  mpi = MpiPartition(ngroups=3)

  equil = Vmec(path, mpi)

# get_LgradB(get_path(960, "nml"))

In [ ]:
for ID in range(1000, 2000):
  simsopt_path = get_path(ID)
  if os.path.exists(simsopt_path):
    soptobj = simsopt.load(simsopt_path)

    lcfs = soptobj[0][-1]
    # XYZ tensor fourier -> RZ fourier
    rzf = lcfs.to_RZFourier()
    curves = [coil.curve for coil in soptobj[1]]
    
    bdistrib_out_path = get_path(ID, "bdistrib")
    subprocess.check_call(["mkdir", "-p", os.path.dirname(bdistrib_out_path)])
    subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf), 
                                                                    sep_outer=minimum_coil_surf_distance(curves, rzf))])
    # Move results to correct directory
    subprocess.check_call(["mv", "bdistrib_out.python_generated.nc", bdistrib_out_path, "-u"])
  else:
    print("Skipping", simsopt_path)

In [ ]:
import bdistribPlot

for ID in range(1000, 2000):
  bdistrib_path = get_path(ID, "bdistrib")
  simsopt_path = get_path(ID, "simsopt")
  if os.path.exists(bdistrib_path):
    print(bdistrib_path)
    bdistribPlot.main(bdistrib_path)
    # soptobj = simsopt.load(simsopt_path)
    # simsopt.geo.plot(soptobj[0] + soptobj[1], engine="plotly")

In [ ]:
tokamak_surf = simsopt.geo.SurfaceRZFourier(nfp=2, stellsym=False, mpol=5, ntor=5)
tokamak_surf.set_zs(1,0, 0.5)
tokamak_surf.set_rc(1,0, 0.4)
tokamak_surf.set_rs(1,0, 0.1)
tokamak_surf.set_rs(2,0, 0.2)


subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(tokamak_surf), 
                                                                sep_outer=0.2)])

bdistribPlot.main("bdistrib_out.tokamak.nc")